Preprocess the data and organise

In [ ]:
from utils.data_process import generate_dataset
from pathlib import Path

# Move to the project for data process 
path_project= #put you own dataset path in string format
%cd #Put your own project path
generate_dataset(Path(path_project),
                 ["fields","roads"])

AUgment data + training to save the model .pt + visualise train/val loss history

In [ ]:
import torch
print(torch.cuda.is_available()) 

#change parameter in the train.py
%run train.py

Load model and evaluate with test data

In [ ]:
from utils.model_utils import load_checkpoint, check_accuracy
from utils.CNN_model import SimpleCNN
from utils.dataset import Field_or_Road
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch


# Hyperparameters (use the same as during training especially image hegh and width )

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_WORKERS = 4
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 336
PIN_MEMORY = True
TEST_IMG_DIR = "dataset_original/test_images" #change to you orwn test data path 
MODEL_NAME = "model_name_best.pt" #change to you own model path

test_transform = A.Compose(
        [
            A.Resize(height = IMAGE_HEIGHT, width = IMAGE_WIDTH),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],     # standard normalisation parameters for ImagNet
                std=[0.229, 0.224, 0.225], 
                max_pixel_value = 255.0,
            ),
            ToTensorV2(),
        ],
    )
test_ds = Field_or_Road(image_dir = TEST_IMG_DIR,
                        transform = test_transform)
test_loader = DataLoader(
    test_ds,
    batch_size= 1,
    num_workers= NUM_WORKERS,
    pin_memory= PIN_MEMORY,
    shuffle= False,
)

model = SimpleCNN(IMAGE_HEIGHT,IMAGE_WIDTH).to(device = DEVICE)
load_checkpoint(torch.load(MODEL_NAME),model)
check_accuracy(test_loader,model,plot=True,device= DEVICE)